Get 30m Data from FXCM

In [1]:
import pandas as pd
import fxcmpy
from pylab import plt
import datetime as dt
import numpy as np
from itertools import repeat 
import os
import sys
%matplotlib inline
plt.style.use('seaborn')

In [2]:
# Set Data Frequency
# The parameter period defines the frequency of the data to be retrieved. 
# minutes: m1, m5, m15 and m30,
# hours: H1, H2, H3, H4, H6 and H8,
# one day: D1,
# one week: W1,
# one month: M1.

request_period='m30'

In [3]:
# make sure output folder for stockdata exist
path='/storage/candles/new_20200123'
data_path=path+'/prices_30m'
if not os.path.isdir(data_path):
    os.mkdir(data_path)

In [4]:
con = fxcmpy.fxcmpy(access_token='032ebf7aafa5e2b96ab167dfaa0b476877099f3f', log_level='error', server='real')

In [5]:
securities = ('AUD/CHF',
              'AUD/JPY',
              'AUD/USD',
              'AUS200',
              'CAD/CHF',
              'CAD/JPY',
              'EUR/AUD',
              'EUR/GBP',
              'EUR/JPY',
              'EUR/NZD',
              'EUR/TRY',
              'GBP/AUD',
              'GBP/CAD',
              'GBP/CHF',
              'GBP/JPY',
              'GBP/NZD',
              'GBP/USD',
              'NAS100',
              'NZD/CAD',
              'NZD/CHF',
              'NZD/JPY',
              'NZD/USD',
              'SPX500',
              'US30',
              'USD/NOK',
              'USD/SEK',
              'XAU/USD',
              )

In [6]:
# Set initial value
start_period = dt.datetime(2010, 7, 1,0,0)
stop_period = dt.datetime(2020, 1, 16, 0,0)
timestep=dt.timedelta(days=30)

In [7]:
for instrument in securities:
    # initialize request window
    start_akt=start_period
    stop_akt=start_akt+timestep
    
    # make sure output folder for prices exist
    dir_name=instrument.replace('/','_')
    price_data_path=data_path+'/'+dir_name
    if not os.path.isdir(price_data_path):
        os.mkdir(price_data_path)
        
    # check if alle prices fetched for this instrument    
    finished_flag=instrument.replace('/','_')+'_finished.csv'    
    if (os.path.exists(price_data_path+'/'+finished_flag) == True):
        print(instrument,' all data fetched')
    else:
        # Get 30 day data blocks and write them to files
        num_files=0
        last_period=False
        while(stop_akt<=stop_period):
            # check if files with requested data exist
            fname=instrument.replace('/','_')+'_data_'+stop_akt.strftime("%Y%m%d")+'.csv'
            if (os.path.exists(price_data_path+'/'+fname) == True):
                    print('file exist: '+fname)
            else:
                cnt=0
                success=False
                while True:
                    try:
                        candle_values=con.get_candles(instrument, period=request_period,start=start_akt, stop=stop_akt)
                        success=True
                    except:
                        con = fxcmpy.fxcmpy(access_token='032ebf7aafa5e2b96ab167dfaa0b476877099f3f', log_level='error', server='real')
                    if(success==True): 
                        break
                    elif(cnt>3):
                        print('Instrument', instrument,'aborted',stop_akt)
                        sys.exit()
                    else:
                        cnt +=1        
                        
                # calculate OHLC
                candle_values['open']=(candle_values['bidopen']+candle_values['askopen'])/2
                candle_values['high']=(candle_values['bidhigh']+candle_values['askhigh'])/2
                candle_values['low']=(candle_values['bidlow']+candle_values['asklow'])/2
                candle_values['close']=(candle_values['bidclose']+candle_values['askclose'])/2
                candle_values=candle_values.drop(columns=['bidopen','bidclose','bidhigh', 'bidlow','askopen', 'askclose',
                                                          'askhigh', 'asklow','tickqty' ])
                # remove duplicates
                candle_values=candle_values.reset_index().drop_duplicates(subset='date',keep='first').set_index('date')
                print(fname+' with shape: '+str(candle_values.shape[0])+' rows | '+str(candle_values.shape[1])+' columns')
                # write to file
                candle_values.to_csv(price_data_path+'/'+fname,index_label='date')
            start_akt=start_akt+timestep
            stop_akt=start_akt+timestep
            num_files +=1
            # adjust last interval
            if((stop_akt>stop_period) and (last_period==False)): 
                stop_akt=stop_period
                last_period=True
               
        finished_df=pd.DataFrame( {'Instrument':[instrument], 'Num files':[num_files]} )
        finished_df.to_csv(price_data_path+'/'+finished_flag)
        print(instrument,'finished with', num_files,'files')
        
        
  

file exist: AUD_CHF_data_20100731.csv
file exist: AUD_CHF_data_20100830.csv
file exist: AUD_CHF_data_20100929.csv
file exist: AUD_CHF_data_20101029.csv
file exist: AUD_CHF_data_20101128.csv
file exist: AUD_CHF_data_20101228.csv
file exist: AUD_CHF_data_20110127.csv
file exist: AUD_CHF_data_20110226.csv
file exist: AUD_CHF_data_20110328.csv
file exist: AUD_CHF_data_20110427.csv
file exist: AUD_CHF_data_20110527.csv
file exist: AUD_CHF_data_20110626.csv
file exist: AUD_CHF_data_20110726.csv
file exist: AUD_CHF_data_20110825.csv
file exist: AUD_CHF_data_20110924.csv
file exist: AUD_CHF_data_20111024.csv
file exist: AUD_CHF_data_20111123.csv
file exist: AUD_CHF_data_20111223.csv
file exist: AUD_CHF_data_20120122.csv
file exist: AUD_CHF_data_20120221.csv
file exist: AUD_CHF_data_20120322.csv
file exist: AUD_CHF_data_20120421.csv
file exist: AUD_CHF_data_20120521.csv
file exist: AUD_CHF_data_20120620.csv
file exist: AUD_CHF_data_20120720.csv
file exist: AUD_CHF_data_20120819.csv
file exist: 

file exist: CAD_CHF_data_20150804.csv
file exist: CAD_CHF_data_20150903.csv
file exist: CAD_CHF_data_20151003.csv
file exist: CAD_CHF_data_20151102.csv
file exist: CAD_CHF_data_20151202.csv
file exist: CAD_CHF_data_20160101.csv
file exist: CAD_CHF_data_20160131.csv
file exist: CAD_CHF_data_20160301.csv
file exist: CAD_CHF_data_20160331.csv
file exist: CAD_CHF_data_20160430.csv
file exist: CAD_CHF_data_20160530.csv
file exist: CAD_CHF_data_20160629.csv
file exist: CAD_CHF_data_20160729.csv
file exist: CAD_CHF_data_20160828.csv
file exist: CAD_CHF_data_20160927.csv
file exist: CAD_CHF_data_20161027.csv
file exist: CAD_CHF_data_20161126.csv
file exist: CAD_CHF_data_20161226.csv
file exist: CAD_CHF_data_20170125.csv
file exist: CAD_CHF_data_20170224.csv
file exist: CAD_CHF_data_20170326.csv
file exist: CAD_CHF_data_20170425.csv
file exist: CAD_CHF_data_20170525.csv
file exist: CAD_CHF_data_20170624.csv
file exist: CAD_CHF_data_20170724.csv
file exist: CAD_CHF_data_20170823.csv
file exist: 

file exist: EUR_GBP_data_20170922.csv
file exist: EUR_GBP_data_20171022.csv
file exist: EUR_GBP_data_20171121.csv
file exist: EUR_GBP_data_20171221.csv
file exist: EUR_GBP_data_20180120.csv
file exist: EUR_GBP_data_20180219.csv
file exist: EUR_GBP_data_20180321.csv
file exist: EUR_GBP_data_20180420.csv
file exist: EUR_GBP_data_20180520.csv
file exist: EUR_GBP_data_20180619.csv
file exist: EUR_GBP_data_20180719.csv
file exist: EUR_GBP_data_20180818.csv
file exist: EUR_GBP_data_20180917.csv
file exist: EUR_GBP_data_20181017.csv
file exist: EUR_GBP_data_20181116.csv
file exist: EUR_GBP_data_20181216.csv
file exist: EUR_GBP_data_20190115.csv
file exist: EUR_GBP_data_20190214.csv
file exist: EUR_GBP_data_20190316.csv
file exist: EUR_GBP_data_20190415.csv
file exist: EUR_GBP_data_20190515.csv
file exist: EUR_GBP_data_20190614.csv
file exist: EUR_GBP_data_20190714.csv
file exist: EUR_GBP_data_20190813.csv
file exist: EUR_GBP_data_20190912.csv
file exist: EUR_GBP_data_20191012.csv
file exist: 

file exist: EUR_TRY_data_20180719.csv
file exist: EUR_TRY_data_20180818.csv
file exist: EUR_TRY_data_20180917.csv
file exist: EUR_TRY_data_20181017.csv
file exist: EUR_TRY_data_20181116.csv
file exist: EUR_TRY_data_20181216.csv
file exist: EUR_TRY_data_20190115.csv
file exist: EUR_TRY_data_20190214.csv
file exist: EUR_TRY_data_20190316.csv
file exist: EUR_TRY_data_20190415.csv
file exist: EUR_TRY_data_20190515.csv
file exist: EUR_TRY_data_20190614.csv
file exist: EUR_TRY_data_20190714.csv
file exist: EUR_TRY_data_20190813.csv
file exist: EUR_TRY_data_20190912.csv
file exist: EUR_TRY_data_20191012.csv
file exist: EUR_TRY_data_20191111.csv
file exist: EUR_TRY_data_20191211.csv
file exist: EUR_TRY_data_20200110.csv
file exist: EUR_TRY_data_20200116.csv
EUR/TRY finished with 117 files
file exist: GBP_AUD_data_20100731.csv
file exist: GBP_AUD_data_20100830.csv
file exist: GBP_AUD_data_20100929.csv
file exist: GBP_AUD_data_20101029.csv
file exist: GBP_AUD_data_20101128.csv
file exist: GBP_AU

file exist: GBP_CHF_data_20190515.csv
file exist: GBP_CHF_data_20190614.csv
file exist: GBP_CHF_data_20190714.csv
file exist: GBP_CHF_data_20190813.csv
file exist: GBP_CHF_data_20190912.csv
file exist: GBP_CHF_data_20191012.csv
file exist: GBP_CHF_data_20191111.csv
file exist: GBP_CHF_data_20191211.csv
file exist: GBP_CHF_data_20200110.csv
file exist: GBP_CHF_data_20200116.csv
GBP/CHF finished with 117 files
file exist: GBP_JPY_data_20100731.csv
file exist: GBP_JPY_data_20100830.csv
file exist: GBP_JPY_data_20100929.csv
file exist: GBP_JPY_data_20101029.csv
file exist: GBP_JPY_data_20101128.csv
file exist: GBP_JPY_data_20101228.csv
file exist: GBP_JPY_data_20110127.csv
file exist: GBP_JPY_data_20110226.csv
file exist: GBP_JPY_data_20110328.csv
file exist: GBP_JPY_data_20110427.csv
file exist: GBP_JPY_data_20110527.csv
file exist: GBP_JPY_data_20110626.csv
file exist: GBP_JPY_data_20110726.csv
file exist: GBP_JPY_data_20110825.csv
file exist: GBP_JPY_data_20110924.csv
file exist: GBP_JP

file exist: GBP_USD_data_20170326.csv
file exist: GBP_USD_data_20170425.csv
file exist: GBP_USD_data_20170525.csv
file exist: GBP_USD_data_20170624.csv
file exist: GBP_USD_data_20170724.csv
file exist: GBP_USD_data_20170823.csv
file exist: GBP_USD_data_20170922.csv
file exist: GBP_USD_data_20171022.csv
file exist: GBP_USD_data_20171121.csv
file exist: GBP_USD_data_20171221.csv
file exist: GBP_USD_data_20180120.csv
file exist: GBP_USD_data_20180219.csv
file exist: GBP_USD_data_20180321.csv
file exist: GBP_USD_data_20180420.csv
file exist: GBP_USD_data_20180520.csv
file exist: GBP_USD_data_20180619.csv
file exist: GBP_USD_data_20180719.csv
file exist: GBP_USD_data_20180818.csv
file exist: GBP_USD_data_20180917.csv
file exist: GBP_USD_data_20181017.csv
file exist: GBP_USD_data_20181116.csv
file exist: GBP_USD_data_20181216.csv
file exist: GBP_USD_data_20190115.csv
file exist: GBP_USD_data_20190214.csv
file exist: GBP_USD_data_20190316.csv
file exist: GBP_USD_data_20190415.csv
file exist: 

file exist: NZD_CHF_data_20181017.csv
file exist: NZD_CHF_data_20181116.csv
file exist: NZD_CHF_data_20181216.csv
file exist: NZD_CHF_data_20190115.csv
file exist: NZD_CHF_data_20190214.csv
file exist: NZD_CHF_data_20190316.csv
file exist: NZD_CHF_data_20190415.csv
file exist: NZD_CHF_data_20190515.csv
file exist: NZD_CHF_data_20190614.csv
file exist: NZD_CHF_data_20190714.csv
file exist: NZD_CHF_data_20190813.csv
file exist: NZD_CHF_data_20190912.csv
file exist: NZD_CHF_data_20191012.csv
file exist: NZD_CHF_data_20191111.csv
file exist: NZD_CHF_data_20191211.csv
file exist: NZD_CHF_data_20200110.csv
file exist: NZD_CHF_data_20200116.csv
NZD/CHF finished with 117 files
file exist: NZD_JPY_data_20100731.csv
file exist: NZD_JPY_data_20100830.csv
file exist: NZD_JPY_data_20100929.csv
file exist: NZD_JPY_data_20101029.csv
file exist: NZD_JPY_data_20101128.csv
file exist: NZD_JPY_data_20101228.csv
file exist: NZD_JPY_data_20110127.csv
file exist: NZD_JPY_data_20110226.csv
file exist: NZD_JP

file exist: SPX500_data_20191211.csv
file exist: SPX500_data_20200110.csv
file exist: SPX500_data_20200116.csv
SPX500 finished with 117 files
file exist: US30_data_20100731.csv
file exist: US30_data_20100830.csv
file exist: US30_data_20100929.csv
file exist: US30_data_20101029.csv
file exist: US30_data_20101128.csv
file exist: US30_data_20101228.csv
file exist: US30_data_20110127.csv
file exist: US30_data_20110226.csv
file exist: US30_data_20110328.csv
file exist: US30_data_20110427.csv
file exist: US30_data_20110527.csv
file exist: US30_data_20110626.csv
file exist: US30_data_20110726.csv
file exist: US30_data_20110825.csv
file exist: US30_data_20110924.csv
file exist: US30_data_20111024.csv
file exist: US30_data_20111123.csv
file exist: US30_data_20111223.csv
file exist: US30_data_20120122.csv
file exist: US30_data_20120221.csv
file exist: US30_data_20120322.csv
file exist: US30_data_20120421.csv
file exist: US30_data_20120521.csv
file exist: US30_data_20120620.csv
file exist: US30_d

file exist: USD_SEK_data_20191111.csv
file exist: USD_SEK_data_20191211.csv
file exist: USD_SEK_data_20200110.csv
file exist: USD_SEK_data_20200116.csv
USD/SEK finished with 117 files
file exist: XAU_USD_data_20100731.csv
file exist: XAU_USD_data_20100830.csv
file exist: XAU_USD_data_20100929.csv
file exist: XAU_USD_data_20101029.csv
file exist: XAU_USD_data_20101128.csv
file exist: XAU_USD_data_20101228.csv
file exist: XAU_USD_data_20110127.csv
file exist: XAU_USD_data_20110226.csv
file exist: XAU_USD_data_20110328.csv
file exist: XAU_USD_data_20110427.csv
file exist: XAU_USD_data_20110527.csv
file exist: XAU_USD_data_20110626.csv
file exist: XAU_USD_data_20110726.csv
file exist: XAU_USD_data_20110825.csv
file exist: XAU_USD_data_20110924.csv
file exist: XAU_USD_data_20111024.csv
file exist: XAU_USD_data_20111123.csv
file exist: XAU_USD_data_20111223.csv
file exist: XAU_USD_data_20120122.csv
file exist: XAU_USD_data_20120221.csv
file exist: XAU_USD_data_20120322.csv
file exist: XAU_US

In [8]:
#candle_values['open'].plot(figsize=(15, 10));